# Homework 2 #
## Word count ##
### Naive version ###

In [1]:
from pyspark import SparkContext, SparkConf

# Spark Setup
conf = SparkConf().setAppName('Word Count').setMaster('local')
sc = SparkContext(conf=conf)

docs = sc.textFile('dataset.txt', 4)  #returns an RDD (there are at least 4 partitions)
print("Number of documents: ", docs.count())

#naive version
words = docs.flatMap(lambda doc: doc.split())
print("Number of words: ", words.count())

couples = words.map(lambda word: (word, 1))\
                .reduceByKey(lambda a,b: a+b)
print("Number of different words: ", couples.count())

Number of documents:  5
Number of words:  56
Number of different words:  32


### Improved version 1 ###

In [3]:
def wordcount(document):
    dictionary = {}  #create empty dictionary to contain (string: integer) couples
    for word in document.split():  #word is a string element from the iterable list given by split()
        if word in dictionary.keys():  #increase the number of occurences
            dictinary[word] = dictionary[word] + 1
        else:
            dictionary[word] = 1
    return [(k,dictionary[k]) for k in dictionary.keys()]  #return a list of couples (string: integer)
                                                           #because returning the dictionary makes Spark angry

couples1 = docs.flatMap(wordcount)\
                .reduceByKey(lambda a,b: a+b)

In [5]:
print("Number of different couples: ", couples1.count())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 18, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\spark-2.3.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 229, in main
  File "C:\opt\spark\spark-2.3.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 224, in process
  File "C:\Users\Riccardo\Anaconda3\lib\site-packages\pyspark\rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\Riccardo\Anaconda3\lib\site-packages\pyspark\rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\Riccardo\Anaconda3\lib\site-packages\pyspark\rdd.py", line 362, in func
    return f(iter